# Combining Data Wrap-up 
If you want to type along with me, use [this notebook](https://humboldt.cloudbank.2i2c.cloud/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fbethanyj0%2Fdata271_sp25&branch=main&urlpath=tree%2Fdata271_sp25%2Flectures%2Fdata271_lec30_live.ipynb) instead. 
If you don't want to type and want to follow along just by executing the cells, stay in this notebook. 

In [ ]:
import numpy as np
import pandas as pd

## Combining Time Series Data

In [ ]:
env_data = pd.DataFrame({
    'time': pd.date_range('2025-04-14 08:00', periods=6, freq='1h'),
    'temperature_C': [14.0, 12.5, 13.0, 13.5, 12.0, 14.2],
    'humidity_pct': [85, 83, 80, 78, 76, 75]
})
env_data.sort_values(by='temperature_C',inplace=True)
env_data

In [ ]:
precip_data = pd.DataFrame({
    'time': pd.date_range('2025-04-14 07:00', periods=9, freq='1h'),
    'precipitation': [0.1, 0.3, 0, 0, 0.1, 0.05, 0.05, 0.1, 0.1],
})
precip_data

In [ ]:
# A standard merge goes in the order of the original left dataframe
pd.merge(env_data, precip_data)

In [ ]:
# merge_ordered sorts the data too
pd.merge_ordered(env_data, precip_data)

In [ ]:
# merge_ordered also allows you to fill in gaps with a method of choice
pd.merge_ordered(env_data, precip_data, fill_method='ffill')

In [ ]:
# When this is most useful?
precip_data = pd.DataFrame({
    'time': pd.date_range('2025-04-14 07:00', periods=10, freq='30min'),
    'precipitation': [0.1, 0.3, 0, 0, 0.1, 0.05, 0.05, 0.1, 0.1, 0.2],
})
precip_data

In [ ]:
# When the frequency of data points is different
env_data

In [ ]:
# When the frequency of data points is different
pd.merge_ordered(env_data, precip_data)

In [ ]:
pd.merge_ordered(env_data, precip_data, fill_method='ffill')

In [ ]:
# Merge_ordered also accepts different "how" args
pd.merge_ordered(env_data, precip_data, how = 'inner')

In [ ]:
# You can also take the approach of "course graining" with resample
hourly_precip = precip_data.resample('1H', on='time').sum()
hourly_precip

In [ ]:
# Merge at course-grained resolution
course = pd.merge_ordered(env_data,hourly_precip.reset_index())
course

In [ ]:
# When should we be careful about resampling?
precip_data['lat'] = [38.1, 38.1, 38.1, 38.1, 38.1, 38.2, 38.2, 38.2, 38.2, 38.2]
precip_data['lon'] = [-120.1, -120.1, -120.2,-120.2, -120.2,-120.1, -120.1, -120.2,-120.2, -120.2]
precip_data

In [ ]:
# Resampling this would lead to inaccurate location data
precip_data.resample('1H',on='time').sum()

In [ ]:
# In this case, use groupby with pd.Grouper
precip_data.groupby(pd.Grouper(key = 'time', freq="1H")).sum()

In [ ]:
# Groupby can handle multiple columns
precip_data.groupby([pd.Grouper(key = 'time', freq="1H"),'lat','lon']).sum()

In [ ]:
# Get course-grained data for each location
precip_data.groupby([pd.Grouper(key = 'time', freq="1H"),'lat','lon']).mean().reset_index()

### More flexible temporal matches

In [ ]:
# Irregular observational data
observations = pd.DataFrame({
    'time': pd.to_datetime([
        '2025-04-14 08:03', '2025-04-14 09:21', '2025-04-14 11:00',
        '2025-04-14 09:45', '2025-04-14 13:27'
    ]),
    'animal': ['fox', 'bear', 'wolf', 'bear', 'wolf'],
    'location': ['ridge', 'valley', 'creek', 'creek', 'ridge']
}).sort_values('time')

observations

In [ ]:
env_data = pd.DataFrame({
    'time': pd.date_range('2025-04-14 08:00', periods=6, freq='1h'),
    'temperature_C': [12.0, 12.5, 13.0, 13.5, 14.0, 14.2],
    'humidity_pct': [85, 83, 80, 78, 76, 75]
})
env_data

In [ ]:
# Merge ordered takes all unique
pd.merge_ordered(observations, env_data)

In [ ]:
# Merge ordered incorrectly duplicates observations
pd.merge_ordered(observations, env_data, fill_method='ffill')

In [ ]:
# Remind ourselves of what the originals look like
observations

In [ ]:
env_data

In [ ]:
# Merge the left time points to the previous point from the right
pd.merge_asof(observations, env_data, on='time')

In [ ]:
# Merge the left time points to the nearest point from the right 
pd.merge_asof(observations, env_data, on='time', direction='nearest')

In [ ]:
# Merge the left time points to the next point from the right
pd.merge_asof(observations, env_data, on='time', direction='forward')

In [ ]:
# If they have different names
env_data.rename(columns={'time':'Time'},inplace=True)
pd.merge_asof(observations, env_data, left_on='time', right_on = 'Time')

In [ ]:
# Reset
env_data.rename(columns={'Time':'time'},inplace=True)

### If we have spatial information too

In [ ]:
observations['lat'] = [38.2, 38.2, 38.2, 38.2, 38.2]
observations['lon'] = [-120.1, -120.1, -120.2,-120.2, -120.2]

In [ ]:
env_data['lat'] = [38.2, 38.2, 38.2, 38.2, 38.2,38.2]
env_data['lon'] = [-120.1, -120.1, -120.2,-120.2, -120.2, -120.1]

In [ ]:
observations

In [ ]:
env_data

In [ ]:
# Merge each location on the time column
pd.merge_asof(observations, env_data, on='time')

In [ ]:
pd.merge_asof(observations, env_data, on='time', by = ['lat','lon'])

This could change the distance it reaches to get a match.

In [ ]:
# Set a tolerance for how close the points have to be to match
pd.merge_asof(observations, env_data, on='time', by = ['lat','lon'],
              tolerance=pd.Timedelta('1h'))

## Activity

You're analyzing bird behavior with GPS trackers and sound-activated recorders. The GPS trackers log positions every 5 minutes. The sound recorders log audio events (calls, songs, other birds) at irregular times. 

For each bird location, merge the most recent sound event recorded in that area.

In [ ]:
bird_gps = pd.DataFrame({
    "time": pd.to_datetime([
        "2023-04-13 08:00", "2023-04-13 08:05", "2023-04-13 08:10"
    ]),
    "bird_id": [101, 101, 101],
    "lat": [40.5, 40.5, 40.5],
    "lon": [-123.8, -123.8, -123.8]
})

sound_events = pd.DataFrame({
    "time": pd.to_datetime([
        "2023-04-13 07:59", "2023-04-13 08:03", "2023-04-13 08:12"
    ]),
    "event_type": ["call", "song", "flight"]
})